In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import plotly.io as pio
pio.renderers.default = "notebook"

sns.set_theme()
plt.rcParams.update({'figure.facecolor':'white'})

import warnings
warnings.filterwarnings("ignore")

# Import results and get a basic overview

In [ ]:
results = pd.read_csv("../statistics/runs.csv")
results.head()

In [ ]:
results['location_id'].nunique()


In [ ]:
results.info()

In [ ]:
for col in [col for col in results.columns if 'prior_scale' in col]:
    results[col] = results[col].replace('None', np.nan).astype(float)
results['changepoint_prior_scale'].fillna(.05, inplace=True)
results['n_changepoints'].fillna(25, inplace=True)
results.info()


In [ ]:
results['location_id'].value_counts()

In [ ]:
results.columns

In [ ]:
parameters = [col for col in results.columns if 'prior_scale' in col]
parameters = parameters + ['n_changepoints']
parameters.sort()
parameters

In [ ]:
results['location_id'].value_counts()

# Clean results

In [ ]:
# Number of locations should be 64. Get all runs with 64 location ID's
df_valid_runs = results.groupby(parameters, dropna=False).count()['Name'].reset_index().rename(columns={'Name': 'count'})
df_valid_runs = df_valid_runs[df_valid_runs['count']==64].drop('count', axis=1)

# get all location ID's from valid runs
valid_locations = []
rows_to_drop = []
for i in df_valid_runs.index:
    for j in results.index:
        if df_valid_runs.loc[i, parameters].astype(float).equals(results.loc[j, parameters].astype(float)):
            valid_locations.append(results['location_id'].loc[j])
valid_locations = set(valid_locations)
print(len(valid_locations))

# get results without duplicates
results_cleaned = results[~results.duplicated(subset=parameters+['location_id'])]
print(results_cleaned.shape)
# filter for valid location ID's
results_cleaned = results_cleaned[results_cleaned['location_id'].isin(valid_locations)]
print(results_cleaned.shape)

In [ ]:
# clean from unfinished runs
df_finished_runs = results_cleaned[parameters]
# count number of experiments per parameter combination
df_finished_runs['count'] = 1
df_finished_runs = df_finished_runs.groupby(parameters, dropna=False).sum().reset_index()
# if that count is equal to the maximum count, it is a finished run
df_finished_runs = df_finished_runs[df_finished_runs['count'] == df_finished_runs['count'].max()]

# compare parameter combinations of all experiments with those of finished runs
# save the indices of experiments of finished runs in a list 
rows_to_keep = []
for i in df_finished_runs.index:
    for j in results_cleaned.index:
        if df_finished_runs.loc[i, parameters].astype(float).equals(results_cleaned.loc[j, parameters].astype(float)):
            rows_to_keep.append(j)

# keep only experiments of finished runs
results_cleaned = results_cleaned.loc[rows_to_keep, :]

# verify by counting experiments per parameter combination
results_cleaned.groupby(parameters, dropna=False).count().iloc[:, 0].reset_index()


# Plot RMSE's

In [ ]:
# plot RMSE's of all locations
fig = px.scatter(
    results_cleaned, 
    x='RMSE_test', 
    y='RMSE_train',
    hover_data=parameters + ['location_id'],
    color='location_id'
)
fig.show()

In [ ]:
# calculate the average RMSE per parameter combination
df_rmse = results_cleaned.groupby(parameters, dropna=False).mean()[['RMSE_test', 'RMSE_train']].reset_index().sort_values('RMSE_test')
df_rmse

In [ ]:
# plot mean RMSE's
fig = px.scatter(
    df_rmse, 
    x='RMSE_test', 
    # y='RMSE_train',
    y='RMSE_train',
    hover_data=parameters,
)
fig.show()

based on RMSE of the test data, the best paramter combination is:
- changepoint_prior_scale   0.60
- humi_prior_scale          NaN (humi flag on False)
- n_changepoints            25.0
- precip_prior_scale        0.3
- press_prior_scale         0.3
- temp_prior_scale          0.3
- winddir_prior_scale       0.3
- windsp_prior_scale        0.3